In [1]:
%load_ext rpy2.ipython

C:\Users\ChengXin\MyProgram\python\anaconda\pro\lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  "The symbol '%s' is not in this R namespace/package." % name


In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import urllib

In [3]:
%%R
data_folder = './'
setwd(data_folder)

### MSstats

In [4]:
%%R -w 800 -h 600
library(MSstats)
library(reticulate)

# 把maxquant数据格式转为out_msstats
#evidence
mq_ev = data.table::fread("evidence.txt")
#proteingroups
#mq_pg = data.table::fread("proteinGroups.txt")
mq_pg = data.table::fread("原-proteinGroups.txt")
#annotation
annot = data.table::fread("2-experimentalDesign.txt")
maxq_imported = MaxQtoMSstatsFormat(mq_ev, annot, mq_pg, use_log_file = FALSE)
#maxq_imported = MaxQtoMSstatsFormat(mq_ev, annot, mq_pg, use_log_file = FALSE, cartesian=TRUE)
fileData <- maxq_imported
# ---------------------

# If run dataProcess() occuring an error message, please change "summaryMethod = 'TMP'" to "summaryMethod = 'linear'"
DDA2009.proposed <- MSstats::dataProcess(raw = fileData,
                                         normalization = 'equalizeMedians',
                                         summaryMethod = 'TMP',
#                                          summaryMethod = 'linear',  
                                         censoredInt = "NA",
                                         MBimpute = TRUE)

# Automatically create the manually created matrix in MSstats, user manual p23
len <- length(levels(DDA2009.proposed$FeatureLevelData$GROUP))

tmp <- t(combn(len,2))
matrix_len = length(t(combn(len,2))) / 2

ourMatrix <- matrix(c(0:0),nrow=matrix_len,ncol=len)

for(i in 1:matrix_len){
  ourMatrix[i, tmp[i]] = -1
  ourMatrix[i, tmp[i + matrix_len]] = 1
}

ourCondition <- levels(DDA2009.proposed$ProteinLevelData$GROUP)
tmp_name <- matrix(ourCondition, nr=len, nc=1)
name <- matrix(nr=matrix_len, nc=1)
for(i in 1:matrix_len){
  name[i,1] <- sprintf('%s-%s', tmp_name[tmp[i+matrix_len]], tmp_name[tmp[i]])
}
row.names(ourMatrix) <- name

#----------End of creation-----------
#names(ourMatrix) <- ourCondition
#ourMatrix.columns <- ourCondition
#colnames(ourMatrix) <- ourCondition


ourMatrix[1] <- 1
ourMatrix[1] <- -1
row.names(ourMatrix) <- "UPS1-UPS2"
ourCondition[1] = "UPS2"
ourCondition[2] = "UPS1"
colnames(ourMatrix) <- ourCondition



DDA2009.comparisons <- MSstats::groupComparison(contrast.matrix = ourMatrix,
                                                data = DDA2009.proposed)


write.csv(DDA2009.comparisons$ComparisonResult, file="MSstats_output_equalizeMedians.csv", index=FALSE)

From cffi callback <function _consolewrite_ex at 0x000001B4BFDB9798>:
Traceback (most recent call last):
  File "C:\Users\ChengXin\MyProgram\python\anaconda\pro\lib\site-packages\rpy2\rinterface_lib\callbacks.py", line 132, in _consolewrite_ex
    s = conversion._cchar_to_str_with_maxlen(buf, n, _CCHAR_ENCODING)
  File "C:\Users\ChengXin\MyProgram\python\anaconda\pro\lib\site-packages\rpy2\rinterface_lib\conversion.py", line 133, in _cchar_to_str_with_maxlen
    s = ffi.string(c, maxlen).decode(encoding)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd4 in position 1: invalid continuation byte
R[write to console]: The following object is masked from 'package:grDevices':

    savePlot




INFO  [2022-09-01 17:40:01] ** Raw data from MaxQuant imported successfully.
INFO  [2022-09-01 17:40:01] ** Rows with values of Potentialcontaminant equal to + are removed 
INFO  [2022-09-01 17:40:02] ** Rows with values of Reverse equal to + are removed 
INFO  [2022-09-01 17:40:02] ** Rows with values of Potentialcontaminant equal to + are removed 
INFO  [2022-09-01 17:40:02] ** Rows with values of Reverse equal to + are removed 
INFO  [2022-09-01 17:40:02] ** Rows with values of Onlyidentifiedbysite equal to + are removed 
INFO  [2022-09-01 17:40:02] ** + Contaminant, + Reverse, + Potential.contaminant, + Only.identified.by.site proteins are removed.
INFO  [2022-09-01 17:40:03] ** Raw data from MaxQuant cleaned successfully.
INFO  [2022-09-01 17:40:03] ** Using provided annotation.
INFO  [2022-09-01 17:40:03] ** Run labels were standardized to remove symbols such as '.' or '%'.
INFO  [2022-09-01 17:40:03] ** The following options are used:
  - Features will be defined by the columns:

In [39]:
MSstats_output_equalizeMedians = pd.read_csv("./MSstats_output_equalizeMedians.csv", header=0, sep=",", index_col=0)
MSstats_output_equalizeMedians.dropna(subset=["log2FC", "pvalue"], how="any", inplace=True)
positive =  MSstats_output_equalizeMedians[(abs(MSstats_output_equalizeMedians["log2FC"]) >1) &(MSstats_output_equalizeMedians["adj.pvalue"] <0.05) ]
negative = MSstats_output_equalizeMedians[(abs(MSstats_output_equalizeMedians["log2FC"]) <=1) | (MSstats_output_equalizeMedians["adj.pvalue"] >=0.05) ]

no_difference_ups = ["P00167", "P01133", "P02144", "P04040", "P15559", "P62937", "P63165",  "Q06830"]

difference_ups =  ["P01579", "O76070", "P63279", "P68871","P01127", "P01008", "P62988", "P10599", "P02787", "P99999", "P12081", "P51965","P10636-8", 
       "P01031", "P09211", "P02788", "P41159", "O00762", "P05413", "P00441", "P00918", "P08758", "P00915", "P01344", "P69905", 
        "P00709", "P55957", "P08263", "P61769", "P10145", "P16083", "P61626", "P02741", "P06732",
        "P01375", "P06396", "P02753", "P01112", "Q15843", "P02768"]

TP = len(positive[positive["Protein"].str.contains("|".join(difference_ups))])
FP = len(positive) - TP

FN = len(negative[negative["Protein"].str.contains("|".join(difference_ups))])
TN = len(negative) - FN

print(TP)
print(FP)
print(TN)
print(FN)

33
4
2179
1


In [40]:
positive

,Protein,Label,log2FC,SE,Tvalue,DF,pvalue,adj.pvalue,issue,MissingPercentage,ImputationPercentage
1,O00762ups|UBE2C_HUMAN_UPS,UPS1-UPS2,4.169775,0.315538,13.214808,5.0,4.433192e-05,0.003276,NaN,0.505682,0.380682
2,O76070ups|SYUG_HUMAN_UPS,UPS1-UPS2,5.944482,0.225745,26.332731,6.0,1.979271e-07,0.000055,NaN,0.386905,0.386905
4,P00441ups|SODC_HUMAN_UPS,UPS1-UPS2,4.580331,0.222107,20.622142,3.0,2.493476e-04,0.014941,NaN,0.519231,0.144231
5,P00709ups|LALBA_HUMAN_UPS,UPS1-UPS2,4.180389,0.203627,20.529617,6.0,8.687620e-07,0.000138,NaN,0.138889,0.138889
6,P00915ups|CAH1_HUMAN_UPS,UPS1-UPS2,-1.822106,0.135916,-13.406111,6.0,1.066679e-05,0.000985,NaN,0.099359,0.099359
7,P00918ups|CAH2_HUMAN_UPS,UPS1-UPS2,-1.860871,0.155656,-11.955032,6.0,2.075360e-05,0.001731,NaN,0.155405,0.155405
8,P01008ups|ANT3_HUMAN_UPS,UPS1-UPS2,6.402380,0.214590,29.835404,6.0,9.402712e-08,0.000035,NaN,0.477041,0.477041
9,P01031ups|CO5_HUMAN_UPS,UPS1-UPS2,-2.039733,0.168701,-12.090823,6.0,1.943973e-05,0.001724,NaN,0.025000,0.025000
11,P01127ups|PDGFB_HUMAN_UPS,UPS1-UPS2,5.256452,0.215592,24.381452,6.0,3.129652e-07,0.000071,NaN,0.391667,0.391667
13,P01344ups|IGF2_HUMAN_UPS,UPS1-UPS2,6.691331,0.353047,18.953098,5.0,7.534177e-06,0.000762,NaN,0.392857,0.267857


In [5]:
%%R -w 800 -h 600

# If run dataProcess() occuring an error message, please change "summaryMethod = 'TMP'" to "summaryMethod = 'linear'"
DDA2009.proposed <- MSstats::dataProcess(raw = fileData,
                                         normalization = 'quantile',
                                         summaryMethod = 'TMP',
#                                          summaryMethod = 'linear',  
                                         censoredInt = "NA",
                                         MBimpute = TRUE)

# Automatically create the manually created matrix in MSstats, user manual p23
len <- length(levels(DDA2009.proposed$FeatureLevelData$GROUP))

tmp <- t(combn(len,2))
matrix_len = length(t(combn(len,2))) / 2

ourMatrix <- matrix(c(0:0),nrow=matrix_len,ncol=len)

for(i in 1:matrix_len){
  ourMatrix[i, tmp[i]] = -1
  ourMatrix[i, tmp[i + matrix_len]] = 1
}

ourCondition <- levels(DDA2009.proposed$ProteinLevelData$GROUP)
tmp_name <- matrix(ourCondition, nr=len, nc=1)
name <- matrix(nr=matrix_len, nc=1)
for(i in 1:matrix_len){
  name[i,1] <- sprintf('%s-%s', tmp_name[tmp[i+matrix_len]], tmp_name[tmp[i]])
}
row.names(ourMatrix) <- name

#----------End of creation-----------
#names(ourMatrix) <- ourCondition
#ourMatrix.columns <- ourCondition
#colnames(ourMatrix) <- ourCondition


ourMatrix[1] <- 1
ourMatrix[1] <- -1
row.names(ourMatrix) <- "UPS1-UPS2"
ourCondition[1] = "UPS2"
ourCondition[2] = "UPS1"
colnames(ourMatrix) <- ourCondition



DDA2009.comparisons <- MSstats::groupComparison(contrast.matrix = ourMatrix,
                                                data = DDA2009.proposed)


write.csv(DDA2009.comparisons$ComparisonResult, file="MSstats_output_quantile.csv")

INFO  [2022-09-02 15:34:02] ** Log2 intensities under cutoff = 19.345  were considered as censored missing values.
INFO  [2022-09-02 15:34:02] ** Log2 intensities = NA were considered as censored missing values.
INFO  [2022-09-02 15:34:02] ** Use all features that the dataset originally has.
INFO  [2022-09-02 15:34:04] 
 # proteins: 2222
 # peptides per protein: 1-142
 # features per peptide: 1-1
INFO  [2022-09-02 15:34:04] Some proteins have only one feature: 
 sp|A5A614|YCIZ_ECOLI,
 sp|P00888|AROF_ECOLI,
 sp|P00903|PABA_ECOLI,
 sp|P03960|KDPB_ECOLI,
 sp|P04391|OTC1_ECOLI;sp|P06960|OTC2_ECOLI ...
INFO  [2022-09-02 15:34:04] 
                    UPS1 UPS2
             # runs    4    4
    # bioreplicates    1    1
 # tech. replicates    4    4
INFO  [2022-09-02 15:34:04] Some features are completely missing in at least one condition:  
 DPAATSVAAAR_2_NA_NA,
 FLTPCYHPNVDTQGNICLDILK_2_NA_NA,
 FLTPCYHPNVDTQGNICLDILK_3_NA_NA,
 LSLEFPSGYPYNAPTVK_2_NA_NA,
 RLQQELM(Oxidation (M))TLM(Oxidation

R[write to console]: Error in write.table(DDA2009.comparisons$ComparisonResult, file = "MSstats_output_quantile.csv",  : 
  unused argument (index = FALSE)




Error in write.table(DDA2009.comparisons$ComparisonResult, file = "MSstats_output_quantile.csv",  : 
  unused argument (index = FALSE)


RInterpreterError: Failed to parse and evaluate line '\n# If run dataProcess() occuring an error message, please change "summaryMethod = \'TMP\'" to "summaryMethod = \'linear\'"\nDDA2009.proposed <- MSstats::dataProcess(raw = fileData,\n                                         normalization = \'quantile\',\n                                         summaryMethod = \'TMP\',\n#                                          summaryMethod = \'linear\',  \n                                         censoredInt = "NA",\n                                         MBimpute = TRUE)\n\n# Automatically create the manually created matrix in MSstats, user manual p23\nlen <- length(levels(DDA2009.proposed$FeatureLevelData$GROUP))\n\ntmp <- t(combn(len,2))\nmatrix_len = length(t(combn(len,2))) / 2\n\nourMatrix <- matrix(c(0:0),nrow=matrix_len,ncol=len)\n\nfor(i in 1:matrix_len){\n  ourMatrix[i, tmp[i]] = -1\n  ourMatrix[i, tmp[i + matrix_len]] = 1\n}\n\nourCondition <- levels(DDA2009.proposed$ProteinLevelData$

In [6]:
%%R

write.csv(DDA2009.comparisons$ComparisonResult, file="MSstats_output_quantile.csv")

In [36]:
MSstats_output_quantile = pd.read_csv("./MSstats_output_quantile.csv", header=0, sep=",", index_col=0)
MSstats_output_quantile.dropna(subset=["log2FC", "pvalue"], how="any", inplace=True)

positive =  MSstats_output_quantile[(abs(MSstats_output_quantile["log2FC"]) >1) &(MSstats_output_quantile["adj.pvalue"] <0.05) ]
negative = MSstats_output_quantile[(abs(MSstats_output_quantile["log2FC"]) <=1) | (MSstats_output_quantile["adj.pvalue"] >=0.05) ]

TP = len(positive[positive["Protein"].str.contains("|".join(difference_ups))])
FP = len(positive) - TP

FN = len(negative[negative["Protein"].str.contains("|".join(difference_ups))])
TN = len(negative) - FN

print(TP)
print(FP)
print(TN)
print(FN)

33
3
2180
1


In [8]:
%%R -w 800 -h 600

# If run dataProcess() occuring an error message, please change "summaryMethod = 'TMP'" to "summaryMethod = 'linear'"
DDA2009.proposed <- MSstats::dataProcess(raw = fileData,
                                         normalization = FALSE,
                                         summaryMethod = 'TMP',
#                                          summaryMethod = 'linear',
                                         censoredInt = "NA",
                                         MBimpute = TRUE)

# Automatically create the manually created matrix in MSstats, user manual p23
len <- length(levels(DDA2009.proposed$FeatureLevelData$GROUP))

tmp <- t(combn(len,2))
matrix_len = length(t(combn(len,2))) / 2

ourMatrix <- matrix(c(0:0),nrow=matrix_len,ncol=len)

for(i in 1:matrix_len){
  ourMatrix[i, tmp[i]] = -1
  ourMatrix[i, tmp[i + matrix_len]] = 1
}

ourCondition <- levels(DDA2009.proposed$ProteinLevelData$GROUP)
tmp_name <- matrix(ourCondition, nr=len, nc=1)
name <- matrix(nr=matrix_len, nc=1)
for(i in 1:matrix_len){
  name[i,1] <- sprintf('%s-%s', tmp_name[tmp[i+matrix_len]], tmp_name[tmp[i]])
}
row.names(ourMatrix) <- name

#----------End of creation-----------
#names(ourMatrix) <- ourCondition
#ourMatrix.columns <- ourCondition
#colnames(ourMatrix) <- ourCondition


ourMatrix[1] <- 1
ourMatrix[1] <- -1
row.names(ourMatrix) <- "UPS1-UPS2"
ourCondition[1] = "UPS2"
ourCondition[2] = "UPS1"
colnames(ourMatrix) <- ourCondition



DDA2009.comparisons <- MSstats::groupComparison(contrast.matrix = ourMatrix,
                                                data = DDA2009.proposed)


write.csv(DDA2009.comparisons$ComparisonResult, file="MSstats_output_FALSE.csv")

INFO  [2022-09-02 15:42:20] ** Log2 intensities under cutoff = 19.332  were considered as censored missing values.
INFO  [2022-09-02 15:42:20] ** Log2 intensities = NA were considered as censored missing values.
INFO  [2022-09-02 15:42:20] ** Use all features that the dataset originally has.
INFO  [2022-09-02 15:42:21] 
 # proteins: 2222
 # peptides per protein: 1-142
 # features per peptide: 1-1
INFO  [2022-09-02 15:42:21] Some proteins have only one feature: 
 sp|A5A614|YCIZ_ECOLI,
 sp|P00888|AROF_ECOLI,
 sp|P00903|PABA_ECOLI,
 sp|P03960|KDPB_ECOLI,
 sp|P04391|OTC1_ECOLI;sp|P06960|OTC2_ECOLI ...
INFO  [2022-09-02 15:42:21] 
                    UPS1 UPS2
             # runs    4    4
    # bioreplicates    1    1
 # tech. replicates    4    4
INFO  [2022-09-02 15:42:22] Some features are completely missing in at least one condition:  
 DPAATSVAAAR_2_NA_NA,
 FLTPCYHPNVDTQGNICLDILK_2_NA_NA,
 FLTPCYHPNVDTQGNICLDILK_3_NA_NA,
 LSLEFPSGYPYNAPTVK_2_NA_NA,
 RLQQELM(Oxidation (M))TLM(Oxidation

In [37]:
MSstats_output_FALSE = pd.read_csv("./MSstats_output_FALSE.csv", header=0, sep=",", index_col=0)
MSstats_output_FALSE.dropna(subset=["log2FC", "pvalue"], how="any", inplace=True)
positive =  MSstats_output_FALSE[(abs(MSstats_output_FALSE["log2FC"]) >1) &(MSstats_output_FALSE["adj.pvalue"] <0.05) ]
negative = MSstats_output_FALSE[(abs(MSstats_output_FALSE["log2FC"]) <=1) | (MSstats_output_FALSE["adj.pvalue"] >=0.05) ]

TP = len(positive[positive["Protein"].str.contains("|".join(difference_ups))])
FP = len(positive) - TP

FN = len(negative[negative["Protein"].str.contains("|".join(difference_ups))])
TN = len(negative) - FN

print(TP)
print(FP)
print(TN)
print(FN)

26
0
2183
8


In [10]:
%%R -w 800 -h 600

# If run dataProcess() occuring an error message, please change "summaryMethod = 'TMP'" to "summaryMethod = 'linear'"
DDA2009.proposed <- MSstats::dataProcess(raw = fileData,
                                         normalization = "equalizeMedians",
                                         summaryMethod = 'TMP',
#                                          summaryMethod = 'linear',
                                         censoredInt = '0',
                                         MBimpute = TRUE)

# Automatically create the manually created matrix in MSstats, user manual p23
len <- length(levels(DDA2009.proposed$FeatureLevelData$GROUP))

tmp <- t(combn(len,2))
matrix_len = length(t(combn(len,2))) / 2

ourMatrix <- matrix(c(0:0),nrow=matrix_len,ncol=len)

for(i in 1:matrix_len){
  ourMatrix[i, tmp[i]] = -1
  ourMatrix[i, tmp[i + matrix_len]] = 1
}

ourCondition <- levels(DDA2009.proposed$ProteinLevelData$GROUP)
tmp_name <- matrix(ourCondition, nr=len, nc=1)
name <- matrix(nr=matrix_len, nc=1)
for(i in 1:matrix_len){
  name[i,1] <- sprintf('%s-%s', tmp_name[tmp[i+matrix_len]], tmp_name[tmp[i]])
}
row.names(ourMatrix) <- name

#----------End of creation-----------
#names(ourMatrix) <- ourCondition
#ourMatrix.columns <- ourCondition
#colnames(ourMatrix) <- ourCondition


ourMatrix[1] <- 1
ourMatrix[1] <- -1
row.names(ourMatrix) <- "UPS1-UPS2"
ourCondition[1] = "UPS2"
ourCondition[2] = "UPS1"
colnames(ourMatrix) <- ourCondition



DDA2009.comparisons <- MSstats::groupComparison(contrast.matrix = ourMatrix,
                                                data = DDA2009.proposed)


write.csv(DDA2009.comparisons$ComparisonResult, file="MSstats_output_inputation0.csv")

INFO  [2022-09-02 15:54:15] ** Log2 intensities under cutoff = 19.422  were considered as censored missing values.
INFO  [2022-09-02 15:54:15] ** Log2 intensities = 0 were considered as censored missing values.
INFO  [2022-09-02 15:54:15] ** Use all features that the dataset originally has.
INFO  [2022-09-02 15:54:17] 
 # proteins: 2222
 # peptides per protein: 1-142
 # features per peptide: 1-1
INFO  [2022-09-02 15:54:17] Some proteins have only one feature: 
 sp|A5A614|YCIZ_ECOLI,
 sp|P00888|AROF_ECOLI,
 sp|P00903|PABA_ECOLI,
 sp|P03960|KDPB_ECOLI,
 sp|P04391|OTC1_ECOLI;sp|P06960|OTC2_ECOLI ...
INFO  [2022-09-02 15:54:17] 
                    UPS1 UPS2
             # runs    4    4
    # bioreplicates    1    1
 # tech. replicates    4    4
INFO  [2022-09-02 15:54:17] Some features are completely missing in at least one condition:  
 DPAATSVAAAR_2_NA_NA,
 FLTPCYHPNVDTQGNICLDILK_2_NA_NA,
 FLTPCYHPNVDTQGNICLDILK_3_NA_NA,
 LSLEFPSGYPYNAPTVK_2_NA_NA,
 RLQQELM(Oxidation (M))TLM(Oxidation 

In [38]:
MSstats_output_inputation0 = pd.read_csv("./MSstats_output_inputation0.csv", header=0, sep=",", index_col=0)
MSstats_output_inputation0.dropna(subset=["log2FC", "pvalue"], how="any", inplace=True)
positive =  MSstats_output_inputation0[(abs(MSstats_output_inputation0["log2FC"]) >1) &(MSstats_output_inputation0["adj.pvalue"] <0.05) ]
negative = MSstats_output_inputation0[(abs(MSstats_output_inputation0["log2FC"]) <=1) | (MSstats_output_inputation0["adj.pvalue"] >=0.05) ]

TP = len(positive[positive["Protein"].str.contains("|".join(difference_ups))])
FP = len(positive) - TP

FN = len(negative[negative["Protein"].str.contains("|".join(difference_ups))])
TN = len(negative) - FN

print(TP)
print(FP)
print(TN)
print(FN)

28
2
2181
6
